## In this note book, objective is to prepare a company brouchure using LLM.
## 1. we use http request to scrap the website data.
## 2. we use beautiful soup to parse the html content to derive, text, emailid, links in any of company website.
## 3. we call mini LLM and provide the links and will ask LLM if any of the links are useful in creating the brouchure.we provide multi-shot prompting to provide more information
## 4. we call another LLM with the text and releavant links to prepare a brouchure for the sales team. 
## 5. we call another LLM to convert the brochure from english to spanish.
## 6. We are calling another LLM to evaluate the effeciency of text conversion from the previous LLM. This is a sample version of Agentic AI solution.



### 1. we use http request to scrap the website data. 
### 2. we use beautiful soup to parse the html content to derive, text, emailid, links in any of company website

In [76]:
import requests
import os
import json
from bs4 import BeautifulSoup
import re 
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import markdown
import pdfkit
import ollama

In [45]:
def scrape_website(url):
    
    headers = {
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }
    try:
        #reading the webpage data using htts request.
        response = requests.get(url, headers = headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return None
    
    text =  response.content
    ##creating an object of beautifulsoup and telling to parse the html data.
    soup = BeautifulSoup(response.content, 'html.parser')
    # Get title
    title = soup.title.string if soup.title else "No title found"
    # Remove unwanted tags
   
    for tag in soup(["script", "style", "img", "input"]):
        tag.decompose()
    # Get visible text
    if soup.body:
        text = soup.get_text(separator='\n', strip=True)    
    else:            
        text = ""

    # Get all links
    links_raw = [a.get('href') for a in soup.find_all('a', href=True)]
  
    # Pattern to match valid https links ending with .com, .net, or .org
    links = [link for link in links_raw if link.startswith("http://") or link.startswith("https://")]

    # Get email address    
    email_pattern = r'(?:mailto:)?([\w\.-]+@[\w\.-]+\.\w+)'
    emails = re.findall(email_pattern, text)
 

    return {
        'title': title,
        'text': text,
        'links': links,
        'emails': emails
     }

In [46]:
url = "https://www.freeman.com/"
webcontent = scrape_website(url)
print(webcontent['title'])

Freeman® | Trade Show, Exhibit, and Event Company


### 3.Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [47]:
# prepare system prompt for the LLM. we are giving oneshot prompt as example for LLM to understand it
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
#link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""


In [48]:
## Now prepare user prompt for the LLM. 
def get_links_user_prompt(url,links):
    user_prompt = f"Here is the list of links on the website of {url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in the below JSON format. \
 with type and respective link Do not include Terms of Service, Privacy, email links.  \n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(links)
    return user_prompt

In [49]:
print(get_links_user_prompt(url,webcontent['links']))

Here is the list of links on the website of https://www.freeman.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in the below JSON format.  with type and respective link Do not include Terms of Service, Privacy, email links.  
Links (some might be relative links):
https://www.freeman.com/
https://www.freeman.com/event-services/av-production/
https://www.freeman.com/event-services/studio-blue/
https://www.freeman.com/event-services/stage-rigging/
https://www.freeman.com/event-services/conferences-trade-shows/
http://www.wearesparks.com
https://www.freeman.com/event-services/exhibitor-services/
https://www.freeman.com/event-services/exhibit-rental-solutions/
https://www.freeman.com/event-services/freeman-transportation/
https://www.freeman.com/event-services/event-technology/
https://www.freeman.com/case-studies/
https://www.freeman.com/resources/
https://www.freeman.com/about-us/
https://www.freeman.com/executiv

## We call LLM with the prompt and let LLM deceide which links are useful to inlcude in the brouchure report

In [50]:
response_format={"type": "json_object"}
input_text = link_system_prompt + get_links_user_prompt(url,webcontent['links'])

In [51]:
print(input_text)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
Here is the list of links on the website of https://www.freeman.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in the below JSON format.  with type and respective link Do not include Terms of Service, Privacy, email links.  
Links (some might be relative links):
https://www.freeman.com/
https://www.freeman.com/event-services/av-production/
https://www.freeman.com/event-services/studio-blue/
https://www.freeman.com/event-services/stage-rigging/
https://www.freeman.com/event-services/conferences-trade

In [52]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [53]:
client = OpenAI()
def llmcall_to_links(input_text):
    response = client.responses.create(
    model="gpt-4.1",
    input= input_text)
    raw_text = response.output[0].content[0].text
    
    clean_text = raw_text.strip('`').replace('json\n', '').replace('```', '')
    url_list = json.loads(clean_text)
    return url_list

In [54]:
url_list = llmcall_to_links(input_text)
for list in url_list:
    print(list)

{'type': 'home page', 'url': 'https://www.freeman.com/'}
{'type': 'about page', 'url': 'https://www.freeman.com/about-us/'}
{'type': 'careers page', 'url': 'https://www.freeman.com/about-us/careers/'}
{'type': 'executive team page', 'url': 'https://www.freeman.com/executive-team/'}
{'type': 'case studies', 'url': 'https://www.freeman.com/case-studies/'}
{'type': 'family of brands', 'url': 'https://www.freeman.com/family-of-brands/'}


## 4. we call another LLM with the text and releavant links to prepare a brouchure for the sales team. 

In [55]:
# prepare instrunctions for the LLM
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, company locations customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [56]:
def get_all_details(url):
    website_contents = scrape_website(url)
    result = "Landing page:\n"
    result += website_contents['text']
    links = llmcall_to_links(input_text)
    #print("Found links:", links)
    for link in links:
        result += f"\n\n{link['type']}\n"
        result += scrape_website(link['url'])['text']
    return result

In [57]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    #user_prompt = user_prompt[:8000] # Truncate if more than 5,000 characters
    return user_prompt

In [58]:
print(get_brochure_user_prompt("Freeman Company","https://www.freeman.com/"))

You are looking at a company called: Freeman Company
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Freeman® | Trade Show, Exhibit, and Event Company
Event Services
AV Production
Studio Blue
Stage Rigging
Conferences & Trade Shows
Corporate Events & Experiences
Exhibits & Environments
Exhibit Rental Solutions
Freeman Transportation®
Event Technology
Case Studies
Resources
About Us
Leadership
Press
Impact
Careers
Family of Brands
Contact Us
Get in Touch
Find My Show
Find Shows & Place Order
Search
We’re more than
a global events company.
News & Insights
Gen Z Report
Why digital natives value live events
Get Study
The Future of Commerce
New Freeman Trends Report findings may surprise you
Read Report
Book:
Inside Live Events
Learn how we make moments
that matter
Order Now
Freeman designs and delivers unforgettable moments
that create a bond between audiences and our clients. It’s 

In [59]:
## calling LLM model to create the brouchure for the company

def llmcall_to_brochure(input_text):
    response = client.responses.create(
    model="gpt-4.1",
    input= input_text)
    raw_text = response.output[0].content[0].text
    return raw_text

In [60]:
company_name = "Freeman Companies"
url = "https://www.freeman.com/"
user_prompt = system_prompt+get_brochure_user_prompt(company_name, url)

result = llmcall_to_brochure(user_prompt)
clean_text = result.strip('`').replace('json\n', '').replace('```', '')
display(Markdown(clean_text))

markdown
# Freeman Companies  
*Designing Experiences. Delivering Connections.*

---

## Who We Are

Freeman is a global leader in the trade show, exhibit, and live events industry. For nearly a century, Freeman has designed and delivered unforgettable moments that create meaningful connections between audiences and the world’s top brands. Our mission is to build loyalty and trust through one-of-a-kind experiences, supported by over 97 years of event expertise and a commitment to innovation.

---

## Our Services

We offer integrated, full-service solutions including:

- **Event Services**
- **AV Production**
- **Stage Rigging**
- **Conferences & Trade Shows**
- **Corporate Events & Experiences**
- **Exhibits & Environments**
- **Exhibit Rental Solutions**
- **Freeman Transportation®**
- **Event Technology**

Each service is designed to maximize audience engagement, strengthen brand experiences, and deliver lasting impressions.

---

## Family of Brands

Freeman’s impact is amplified by a diverse family of brands, bringing specialist expertise to every moment:

- **Alford Media** – Audio, video, lighting, visual effects, webcasting, and event technology
- **BaAM Productions** – High-profile sports and entertainment experiences
- **mdg** – Marketing agency and advisory partner for events organizations and associations
- **Sparks** – Global experiential marketing, crafting live and digital brand experiences
- **Studio Blue** – Creative AV production designed to engage and entertain

---

## Signature Work & Clients

We help the world’s leading organizations deliver memorable experiences:

- **Hard Rock** — Rock star presence across multiple shows
- **RE+ 2024** — Enhanced attendee navigation
- **AIA** — Building connections within the architecture industry
- **Groceryshop** — Comprehensive AV solutions
- **Avis** — Generating excitement at critical industry shows
- **NATO Summit** — Celebrating 75 years of alliance

---

## Company Culture & Values

We are united by values that support our drive to create “moments that matter”:

- **Integrity** – Honesty, trust, and doing the right thing above all
- **Empathy** – Putting people first, understanding needs and perspectives
- **Innovation** – Relentlessly seeking ideas that deliver value and improvement
- **Enthusiasm** – Confidence and optimism in every challenge
- **Learning Mindset** – Fueling innovation through curiosity and continuous learning

Our commitment to **Diversity, Equity & Inclusion**, as well as sustainability through our Net Zero Carbon Events Pledge, underscores our leadership in responsible business practices.

---

## Global Presence

Freeman is headquartered in Dallas, Texas, with operations across the United States, Canada, the United Kingdom, China, Singapore, and beyond, enabling us to serve customers worldwide.

---

## Leadership

Our forward-thinking executive team ensures innovation and client focus are at the heart of everything we do:

- **Janet Dell** – CEO & Board Member
- Supported by a talented leadership in finance, operations, technology, legal, marketing, and people development

---

## Careers at Freeman

Experience the thrilling world of live events. Join a company that values:

- Collaboration
- Curiosity and growth
- Making a real impact for clients and communities

**Benefits include:**  
Health insurance, life/accident/disability plans, tuition reimbursement, retirement matches, paid parental leave, and generous paid time off.

*Students and early-career professionals are welcome through robust internship opportunities.*

---

## Contact Us

Ready to create your next unforgettable experience?  
**Exhibitor & Event Support:**  
- **US & Canada:** (888) 508-5054  
- **International:** +1 (817) 210-4869  
- **Freeman Transportation:** US (800) 995-3579 | Canada (877) 478-1113  
- **General Inquiries:** [Contact Us](mailto:[email protected])

Discover more:  
[Freeman LinkedIn](https://linkedin.com/company/freeman)  
[Freeman Instagram](https://instagram.com/freeman_co)

---

_Freeman Companies: Crafting moments that matter for today and the future._



## Calling LLM with chat completion instead of response create. 

In [61]:
company_name = "Freeman Companies"
url = "https://www.freeman.com/"
model="gpt-4.1"
openai = OpenAI()
pdf_path = "week1/freeman_brochure.pdf" 

In [66]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    clean_text = result.strip('`').replace('json\n', '').replace('```', '')
    return((clean_text))


In [67]:
brochure_text = create_brochure(company_name, url)
print(brochure_text)

markdown
# Freeman®  
**Moments That Matter – Unforgettable Events, Global Connections**

---

## Who We Are

Freeman is a global leader in live events, exhibits, and experiential marketing solutions. With almost a century of experience and a future-focused mindset, we design and deliver experiences that create meaningful connections between our clients and their audiences. Our integrated, full-service solutions, industry-leading technology, and award-winning teams ensure that every event is both unforgettable and impactful.

---

## What We Do

- **Event Services, AV Production & Stage Rigging**  
  From conferences and trade shows to dynamic stage setups, we deliver seamless production solutions.

- **Corporate Events & Brand Experiences**  
  We elevate brands by designing interactive experiences that build loyalty and trust.

- **Exhibits & Environments / Rentals**  
  Our custom and rental exhibits are designed to captivate, engage, and inspire.

- **Freeman Transportation®**  
  

## Calling LLM Model with streams of data 

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure(company_name, url)

### Now calling another LLM to convert the brouchure text generated in English to French. For this again we need to create the system prompt and user prompt before we call an LLM Model


In [85]:
# prepare instrunctions for the LLM
system_prompt = "You are a language translator and your job is to translate the text of document from English to French. \
Don't use any abusive language and keep the meaning of the text. Keep the same formating/n"

In [86]:
user_prompt = " Below is the brouchure text of a company called {company_name} and its website url is {url} in English. Convert it to French.\n"
user_prompt += brochure_text

In [87]:
## Calling LLM Model from OLLAMA

In [93]:
def messages_for():
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [94]:

model = "llama3.2:latest"
messages = messages_for()


In [ ]:
response = ollama.chat(model=model, messages=messages)
french_brochure = response['message']['content']
display(Markdown(french_brochure))


In [ ]:
## to evaluate the conversion of English to French. 
##we call again another LLM to validate and provide feedback on whether the language translation was accurate or not.
## This is another example fo Agentic- AI where we are calling another LLM to evaluate the output of anotherLLM.

In [118]:
# prepare instrunctions for the LLM
system_prompt = (
    "You are a language translation evaluator. Your task is to assess the accuracy of French translations generated by an LLM "
    "from English source texts. You will be given an English sentence along with its translated French version. \n\n"
    "Evaluate the translation and return an accuracy score (in percentage) indicating how effectively the English text was translated into French.\n\n"
    "If the accuracy is below 95%, generate a new French translation for the given English sentence. Then re-evaluate the new translation.\n\n"
    "Repeat this process of translation and evaluation until the accuracy exceeds 95%. When that happens, return the final French translation "
    "along with the achieved accuracy."
)


In [119]:
user_prompt = "Below is the original English version of text /n"
user_prompt += brochure_text 
user_prompt += "/nNow Below is the converted text to French provided by another LLM/n"
user_prompt += french_brochure
user_prompt += "/n Now evaluate the conversion to French and just provide an accuracy percentage only"

In [120]:
def messages_for():
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [121]:
model = "gemma3:latest"
messages = messages_for()

In [122]:
response = ollama.chat(model=model, messages=messages)
traslate_evaluate = response['message']['content']
display(Markdown(traslate_evaluate ))

92%
